# Imports

In [14]:
import os
import random
import pandas as pd
from pydub import AudioSegment
from midi2audio import FluidSynth
from mido import Message, MidiFile, MidiTrack
import pygame
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import time
import cv2

midi_file = 'random.mid'
wave_file = 'output.wav'

# Note map

In [15]:
note_map = [45, 47, 48, 50, 52, 53, 55]

# Generator

In [16]:
def generate_midi(midi_file: str, note_map: dict, length=32, interval=500):
    midi = MidiFile()
    track = MidiTrack()
    midi.tracks.append(track)
    note_count = len(note_map)
    

    active = np.zeros(note_count, dtype=np.byte)
    cooldown = 0
    for _ in range(length):
        cooldown += interval
        off_notes = (np.random.rand(note_count) < 0.2).astype(np.byte) & active
        for i in np.where(off_notes)[0]:
            track.append(Message('note_on', note=note_map[int(i)], velocity=0, time=cooldown))
            active[i] = 0
            cooldown = 0

        on_notes = (np.random.rand(note_count) < 0.1).astype(np.byte) & ~active
        for i in np.where(on_notes)[0]:
            track.append(Message('note_on', note=note_map[int(i)], velocity=100, time=cooldown))
            active[i] = 1
            cooldown = 0

    cooldown = interval
    for i in np.where(active)[0]:
        track.append(Message('note_on', note=note_map[int(i)], velocity=0, time=cooldown))
        active[i] = 0
        cooldown = 0

    midi.save(midi_file)

# Example

In [20]:
generate_midi(midi_file, note_map, length=16, interval=300)

In [21]:
pygame.init()
pygame.mixer.init()
pygame.mixer.music.load(midi_file)
pygame.mixer.music.play()

In [19]:
# output_dir = 'my_midi'
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# files = []
# for i in range(100):
#     mf = f'{output_dir}/{i}.mid'
#     generate_midi(mf, note_map, length=1024, interval=300)
#     files.append(mf)

# pd.DataFrame(files, columns=['midi_filename']).to_csv('midi_files.csv', index=False)
